In [31]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql.functions import *

In [18]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName('test') \
        .getOrCreate()

In [19]:
df = spark.read \
    .option("header", "true") \
    .csv("fhvhv_tripdata_2021-06.csv.gz")

In [20]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.IntegerType(), True)
])

In [21]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-06.csv.gz')

df = df.repartition(12)

df.write.parquet('task_2')

23/03/03 22:35:15 WARN TaskSetManager: Lost task 0.0 in stage 4.0 (TID 4) (dainstudios executor driver): TaskKilled (Stage cancelled)


23/03/03 22:35:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/03/03 22:35:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/03/03 22:35:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/03/03 22:35:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/03/03 22:35:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers


23/03/03 22:35:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/03/03 22:35:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/03/03 22:35:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/03/03 22:35:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


Question 3

In [27]:
df.select('*').where((df.pickup_datetime >= '2021-06-15') & (df.pickup_datetime < '2021-06-16')).count()

452470

Question 4

In [35]:
df.withColumn('DiffInSeconds',col("dropoff_datetime").cast("long") - col('pickup_datetime').cast("long")).withColumn("DiffInHours",col("DiffInSeconds")/3600) \
.orderBy(col("DiffInHours").desc()).show(truncate=False)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-------------+------------------+
|dispatching_base_num|pickup_datetime    |dropoff_datetime   |PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|DiffInSeconds|DiffInHours       |
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-------------+------------------+
|B02872              |2021-06-25 13:55:41|2021-06-28 08:48:25|98          |265         |N      |null                  |240764       |66.8788888888889  |
|B02765              |2021-06-22 12:09:45|2021-06-23 13:42:44|188         |198         |N      |null                  |91979        |25.549722222222222|
|B02879              |2021-06-27 10:32:29|2021-06-28 06:31:20|78          |169         |N      |null                  |71931        |19.980833333333333|
|B02800              |2021-06-26 22:37:11|2021-06-27 16:49:01|263         |36     

In [36]:
df2 = spark.read \
    .option("header", "true") \
    .csv("taxi_zone_lookup.csv")

In [37]:
df2.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [39]:
df3 = df.join(df2,df.PULocationID ==  df2.LocationID,"inner")

In [47]:
df3.groupBy("Zone").count().orderBy(col("count").desc()).show(truncate=False)

+-------------------------+------+
|Zone                     |count |
+-------------------------+------+
|Crown Heights North      |231279|
|East Village             |221244|
|JFK Airport              |188867|
|Bushwick South           |187929|
|East New York            |186780|
|TriBeCa/Civic Center     |164344|
|LaGuardia Airport        |161596|
|Union Sq                 |158937|
|West Village             |154698|
|Astoria                  |152493|
|Lower East Side          |151020|
|East Chelsea             |147673|
|Central Harlem North     |146402|
|Williamsburg (North Side)|143683|
|Park Slope               |143594|
|Stuyvesant Heights       |141427|
|Clinton East             |139611|
|West Chelsea/Hudson Yards|139431|
|Bedford                  |138428|
|Murray Hill              |137879|
+-------------------------+------+
only showing top 20 rows

